In [1]:
import cantools
import can 
import pandas as pd
import os
import random
import numpy as np
from scipy.stats import norm
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
pd.set_option('display.float_format', '{:.6f}'.format)
# 设置显示选项
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.width', None)  # 自动调整宽度

指定文件以后，删除无序部分，然后再截取一部分注入ddos攻击。这个模块也不需要保留。不用将数据保存为csv

In [5]:
import os
import pandas as pd

def process_and_split_dataframe(df, remove_fraction=0.2, output_directory="./output"):
    """
    去掉 DataFrame 后面指定比例的行，并将剩下的部分切分成两个等分的 DataFrame。

    参数:
    df (pd.DataFrame): 输入的 DataFrame。
    remove_fraction (float): 要去掉的行的比例（0 到 1 之间）。
    output_directory (str): 保存结果的输出目录。
    """
    # 确保输出目录存在
    os.makedirs(output_directory, exist_ok=True)

    # 计算要去掉的行数
    num_rows = len(df)
    num_rows_to_remove = int(num_rows * remove_fraction)

    # 去掉后面的行
    df_remaining = df.iloc[:-num_rows_to_remove] if num_rows_to_remove > 0 else df

    # 切分 DataFrame 为两个等分
    mid_index = len(df_remaining) - len(df_remaining) // 5
    df_part1 = df_remaining.iloc[:mid_index]
    # df_part2 = df_remaining.iloc[mid_index//2:mid_index]
    df_part3 = df_remaining.iloc[mid_index:]

    # 保存到指定目录
    df_part1.to_csv(os.path.join(output_directory, "part1.csv"), index=False)
    # df_part2.to_csv(os.path.join(output_directory, "part2.csv"), index=False)
    df_part3.to_csv(os.path.join(output_directory, "part3.csv"), index=False)

# 使用示例
input_file_path = r"./CH0_2024-03-26_14-26-10.blf.csv"   # 输入文件路径
output_directory = r"./data523/"  # 输出路径

# 使用示例
# input_file_path = r"./data_processed/can0/CH0_2024-03-26_14-26-10.blf.csv"   # 输入文件路径
# output_directory = r"./data_processed/can0_final"  # 输出路径
# 读取输入文件
df = pd.read_csv(input_file_path)

# 处理和切分 DataFrame
process_and_split_dataframe(df, remove_fraction=0.2, output_directory=output_directory)

In [5]:
# 切分df为小块
def split_dataframe_and_save(input_directory, output_directory, chunk_size=1000):
    """
    将指定目录中的每个 CSV 文件切分为多个 DataFrame，并保存到输出目录。

    参数:
    input_directory (str): 输入 CSV 文件的目录路径。
    output_directory (str): 保存切分后 DataFrame 的目录路径。
    chunk_size (int): 每个切片的行数。
    """
    # 确保输出目录存在
    os.makedirs(output_directory, exist_ok=True)

    # 遍历输入目录中的所有 CSV 文件
    for filename in os.listdir(input_directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_directory, filename)
            df = pd.read_csv(file_path)

            # 切分 DataFrame
            for i in range(0, len(df), chunk_size):
                chunk_df = df.iloc[i:i + chunk_size]
                # 生成新的文件名，包含原文件名和切片索引
                new_filename = f"{os.path.splitext(filename)[0]}_chunk_{i // chunk_size + 1}.csv"
                output_file_path = os.path.join(output_directory, new_filename)

                # 保存切片到 CSV 文件
                chunk_df.to_csv(output_file_path, index=False)

def split_single_file_and_save(input_file_path, output_directory, chunk_size=1000):
    """
    将指定的单个 CSV 文件切分为多个 DataFrame，并保存到输出目录。

    参数:
    input_file_path (str): 输入 CSV 文件的路径。
    output_directory (str): 保存切分后 DataFrame 的目录路径。
    chunk_size (int): 每个切片的行数。
    """
    # 确保输出目录存在
    os.makedirs(output_directory, exist_ok=True)

    # 读取指定的 CSV 文件
    df = pd.read_csv(input_file_path)

    # 切分 DataFrame
    for i in range(0, len(df), chunk_size):
        chunk_df = df.iloc[i:i + chunk_size]
        # 生成新的文件名，包含原文件名和切片索引
        new_filename = f"{os.path.splitext(os.path.basename(input_file_path))[0]}_chunk_{i // chunk_size + 1}.csv"
        output_file_path = os.path.join(output_directory, new_filename)

        # 保存切片到 CSV 文件
        chunk_df.to_csv(output_file_path, index=False)


In [6]:
def split_with_sliding_window(input_file_path, output_directory, window_size=100, step_size=10):
    """
    使用滑动窗口将指定的单个CSV文件切分为多个DataFrame，并保存到输出目录。
    每个窗口会有重叠部分(除非step_size等于window_size)。

    参数:
    input_file_path (str): 输入CSV文件的路径
    output_directory (str): 保存切分后DataFrame的目录路径
    window_size (int): 每个窗口的行数
    step_size (int): 滑动窗口的步长(每次移动的行数)
    """
    # 确保输出目录存在
    os.makedirs(output_directory, exist_ok=True)

    # 读取CSV文件
    df = pd.read_csv(input_file_path)
    total_rows = len(df)
    
    # 检查参数有效性
    if window_size <= 0 or step_size <= 0:
        raise ValueError("window_size和step_size必须大于0")
    if window_size > total_rows:
        raise ValueError("window_size不能大于文件总行数")
    
    # 计算窗口数量
    num_windows = (total_rows - window_size) // step_size + 1
    
    # 生成滑动窗口
    for i in range(num_windows):
        start = i * step_size
        end = start + window_size
        window_df = df.iloc[start:end]
        
        # 生成新文件名
        base_name = os.path.splitext(os.path.basename(input_file_path))[0]
        new_filename = f"{base_name}_window_{i+1}_rows_{start}_{end}.csv"
        output_file_path = os.path.join(output_directory, new_filename)
        
        # 保存窗口数据
        window_df.to_csv(output_file_path, index=False)
    
    # 处理剩余数据(如果step_size不能整除(total_rows - window_size))
    remaining = (total_rows - window_size) % step_size
    if remaining > 0 and num_windows > 0:  # 确保至少有一个窗口
        last_window_df = df.iloc[-window_size:]
        new_filename = f"{base_name}_window_{num_windows+1}_rows_{total_rows-window_size}_{total_rows}.csv"
        output_file_path = os.path.join(output_directory, new_filename)
        last_window_df.to_csv(output_file_path, index=False)


In [33]:
# 拆分成100的小块
input_file_path = r"./data523/part1.csv"  # 输入文件路径
output_directory = r"./data523/can0_100s_train/normal"  # 输出路径
split_with_sliding_window(input_file_path, output_directory, window_size=500, step_size=50)

### DDoS攻击注入：洪水攻击，数据处理

In [24]:
# def inject_ddos_attack(df, num_attacks):
#     """
#     在 DataFrame 中注入 DDoS 攻击数据。

#     参数:
#     df (pd.DataFrame): 原始 DataFrame。
#     num_attacks (int): 注入的攻击数据数量。

#     返回:
#     pd.DataFrame: 注入攻击数据后的 DataFrame。
#     """
#     # 获取原始数据的时间戳范围
#     start_time = df['timestamp'].min()
#     end_time = df['timestamp'].max()

#     # 生成随机的攻击数据
#     attack_rows = []  # 创建一个列表以存储攻击数据行
#     for _ in range(num_attacks):
#         timestamp = random.uniform(start_time, end_time)  # 随机时间戳
#         arbitration_id = 0 # 随机选择一个 arbitration_id=0
#         data = np.random.bytes(8).hex()  # 生成随机的 8 字节数据

#         # 创建攻击数据行
#         attack_row = {
#             "timestamp": timestamp,
#             "arbitration_id": arbitration_id,
#             "data": data
#         }
#         attack_rows.append(attack_row)  # 将每一行添加到列表中

#     # 将所有攻击行转换为 DataFrame 并与原始 DataFrame 连接
#     attack_df = pd.DataFrame(attack_rows)
#     df = pd.concat([df, attack_df], ignore_index=True)

#     # 根据 timestamp 重新排序 DataFrame
#     df.sort_values(by='timestamp', inplace=True)

#     return df
def inject_ddos_attack(df, chunk_size=100, attack_ratio=0.1):
    """
    分块注入DDoS攻击数据，保证每chunk_size行至少注入attack_ratio比例的攻击数据
    
    参数:
    df (pd.DataFrame): 原始数据框
    chunk_size (int): 分块大小
    attack_ratio (float): 攻击数据比例（0.0~1.0）
    
    返回:
    pd.DataFrame: 注入攻击后的数据框
    """
    # 初始化攻击数据容器
    attack_dfs = []
    
    # 分块处理原始数据
    for chunk_start in range(0, len(df), chunk_size):
        chunk = df.iloc[chunk_start:chunk_start + chunk_size]
        
        # 计算本块需要注入的攻击数量（至少1条）
        num_attacks = max(int(len(chunk) * attack_ratio), 1)
        
        # 获取本块时间范围
        start_time = chunk['timestamp'].min()
        end_time = chunk['timestamp'].max()
        
        # 生成攻击数据
        attack_rows = [{
            "timestamp": random.uniform(start_time, end_time),
            "arbitration_id": 0,
            "data": np.random.bytes(8).hex()
        } for _ in range(num_attacks)]
        
        attack_dfs.append(pd.DataFrame(attack_rows))
    
    # 合并所有攻击数据
    all_attack_df = pd.concat(attack_dfs)
    
    # 合并原始数据并重新排序
    return pd.concat([df, all_attack_df]).sort_values('timestamp').reset_index(drop=True)

def process_single_file_ddos_injection(input_file_path, output_ddos_path, chunk_size=100, attack_fraction=0.1, seed=None):
    """
    对指定的单个文件进行 DDoS 攻击注入，并保存结果。

    参数:
    input_file_path (str): 输入 CSV 文件的路径。
    output_ddos_path (str): 保存 DDoS 注入结果的文件路径。
    attack_fraction (float): 注入的 DDoS 攻击数据比例。
    seed (int): 随机种子，用于确定随机选择的一致性。
    """
    # 设置随机种子
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # 读取输入文件
    df = pd.read_csv(input_file_path)
    
    # 执行注入
    injected_df = inject_ddos_attack(df, chunk_size, attack_fraction)
    
    # 保存结果
    os.makedirs(os.path.dirname(output_ddos_path), exist_ok=True)
    injected_df.to_csv(output_ddos_path, index=False)

    # # 计算要注入的攻击数据数量
    # num_attacks = int(len(df) * attack_fraction)  # 假设注入指定比例的攻击数据

    # # 注入 DDoS 攻击数据
    # df_ddos = inject_ddos_attack(df, num_attacks)

    # # 保存注入结果
    # df_ddos.to_csv(output_ddos_path, index=False)


# # 使用示例
# input_directory = r"./data_processed/can0_1000"  # 输入路径
# ddos_directory = r"./data_processed/can0_ddos_30"  # DDoS 注入结果路径
# normal_directory = r"./data_processed/can0_normal"  # 正常文件路径
# process_ddos_injection(input_directory, ddos_directory, normal_directory, seed=2023)

注入ddos攻击

In [52]:
input_file_path = r"./data523/part1.csv"  # 输入文件路径
output_ddos_path = r"./data523/ddos_injected_file.csv"  # DDoS 注入结果文件路径
process_single_file_ddos_injection(input_file_path, output_ddos_path, chunk_size=500, attack_fraction=0.3, seed=2023)

然后切割ddos文件

In [53]:
input_file_path = r"./data523/ddos_injected_file.csv"  # 输入文件路径
output_directory = r"./data523/can0_100s_train/ddos"  # 输出路径
split_with_sliding_window(input_file_path, output_directory, window_size=500, step_size=50)

给测试代码注入ddos

In [54]:
input_file_path = r"./data523/part3.csv"  # 输入文件路径
output_ddos_path = r"./data523/ddos_injected_test.csv"  # DDoS 注入结果文件路径
process_single_file_ddos_injection(input_file_path, output_ddos_path, chunk_size=500, attack_fraction=0.3, seed=2023)

然后切割测试数据

In [11]:
input_file_path = r"./data523/ddos_injected_test.csv"  # 输入文件路径
output_directory = r"./data523/can0_500s_test/ddos"  # 输出路径
split_with_sliding_window(input_file_path, output_directory, window_size=500, step_size=50)


In [10]:

input_file_path = r"./data523/part3.csv"  # 输入文件路径
output_directory = r"./data523/can0_500s_test/normal"  # 输出路径
split_with_sliding_window(input_file_path, output_directory, window_size=500, step_size=50)

### 重放攻击注入

In [2]:
def inject_replay_attack(
    df,
    chunk_size=100,
    attack_ratio=0.1,
    num_attack_ids=1,  # 默认1个攻击ID（即原来的行为，ID=0）
    attack_ids=None,   # 默认None，即不指定攻击ID
):
    """
    分块注入DDoS攻击数据，保证每chunk_size行至少注入attack_ratio比例的攻击数据
    
    参数:
    df (pd.DataFrame): 原始数据框
    chunk_size (int): 分块大小
    attack_ratio (float): 攻击数据比例（0.0~1.0）
    num_attack_ids (int): 使用的攻击ID数量（默认1，即保持原行为）
    attack_ids (list): 指定要使用的攻击ID列表（默认None，即使用ID=0）
    
    返回:
    pd.DataFrame: 注入攻击后的数据框
    """
    # 初始化攻击数据容器
    attack_dfs = []
    
    # 确定攻击ID列表
    if attack_ids is None:
        if num_attack_ids == 1:
            # 默认行为：仅使用ID=0
            attack_ids = [0]
        else:
            # 自动选择num_attack_ids个攻击ID（排除0）
            existing_ids = set(df['arbitration_id'].unique())
            if 0 in existing_ids:
                existing_ids.remove(0)
            
            # 如果现有ID不足，补充随机ID
            if len(existing_ids) < num_attack_ids:
                additional_ids = set()
                while len(additional_ids) < (num_attack_ids - len(existing_ids)):
                    new_id = random.randint(1, 0x7FF)  # 标准CAN ID范围
                    if new_id not in existing_ids:
                        additional_ids.add(new_id)
                existing_ids.update(additional_ids)
            
            attack_ids = random.sample(list(existing_ids), min(num_attack_ids, len(existing_ids)))
    else:
        # 使用用户指定的攻击ID，但不超过num_attack_ids个
        attack_ids = attack_ids[:num_attack_ids]
    
    # 分块处理原始数据
    for chunk_start in range(0, len(df), chunk_size):
        chunk = df.iloc[chunk_start:chunk_start + chunk_size]
        
        # 计算本块需要注入的攻击数量（至少1条）
        num_attacks = max(int(len(chunk) * attack_ratio), 1)
        
        # 获取本块时间范围
        start_time = chunk['timestamp'].min()
        end_time = chunk['timestamp'].max()
        
        # 生成攻击数据
        attack_rows = []
        for _ in range(num_attacks):
            attack_id = random.choice(attack_ids)  # 从攻击ID列表中随机选一个
            attack_rows.append({
                "timestamp": random.uniform(start_time, end_time),
                "arbitration_id": attack_id,
                "data": np.random.bytes(8).hex(),
            })
        
        attack_dfs.append(pd.DataFrame(attack_rows))
    
    # 合并所有攻击数据
    all_attack_df = pd.concat(attack_dfs)
    
    # 合并原始数据并重新排序
    return pd.concat([df, all_attack_df]).sort_values('timestamp').reset_index(drop=True)


def process_single_file_ddos_injection(
    input_file_path,
    output_ddos_path,
    chunk_size=100,
    attack_fraction=0.1,
    num_attack_ids=1,  # 默认1个攻击ID（即原来的行为，ID=0）
    attack_ids=None,   # 默认None，即不指定攻击ID
    seed=None,
):
    """
    对指定的单个文件进行 DDoS 攻击注入，并保存结果。

    参数:
    input_file_path (str): 输入 CSV 文件的路径。
    output_ddos_path (str): 保存 DDoS 注入结果的文件路径。
    chunk_size (int): 分块大小
    attack_fraction (float): 注入的 DDoS 攻击数据比例。
    num_attack_ids (int): 使用的攻击ID数量（默认1，即保持原行为）
    attack_ids (list): 指定要使用的攻击ID列表（默认None，即使用ID=0）
    seed (int): 随机种子，用于确定随机选择的一致性。
    """
    # 设置随机种子
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # 读取输入文件
    df = pd.read_csv(input_file_path)
    
    # 执行注入
    injected_df = inject_replay_attack(
        df,
        chunk_size=chunk_size,
        attack_ratio=attack_fraction,
        num_attack_ids=num_attack_ids,
        attack_ids=attack_ids,
    )
    
    # 保存结果
    os.makedirs(os.path.dirname(output_ddos_path), exist_ok=True)
    injected_df.to_csv(output_ddos_path, index=False)

In [18]:
# 使用指定的攻击ID列表（如[0x123, 0x456]）
input_file_path = r"./data523/part1.csv"  # 输入文件路径
output_ddos_path = r"./data523/replay_injected_file.csv"  # DDoS 注入结果文件路径
process_single_file_ddos_injection(input_file_path, output_ddos_path, attack_fraction=0.3, chunk_size=500, num_attack_ids=1, attack_ids=[0x257])

In [19]:
input_file_path = r"./data523/replay_injected_file.csv"  # 输入文件路径
output_directory = r"./data523/can0_100s_train/replay"  # 输出路径
split_with_sliding_window(input_file_path, output_directory, window_size=500, step_size=50)

In [20]:
# 使用指定的攻击ID列表（如[0x123, 0x456]）test
input_file_path = r"./data523/part3.csv"  # 输入文件路径
output_ddos_path = r"./data523/replay_injected_test.csv"  # DDoS 注入结果文件路径
process_single_file_ddos_injection(input_file_path, output_ddos_path, attack_fraction=0.3, chunk_size=500, num_attack_ids=1, attack_ids=[0x257])

In [21]:
input_file_path = r"./data523/replay_injected_test.csv"  # 输入文件路径
output_directory = r"./data523/can0_500s_test/replay"  # 输出路径
split_with_sliding_window(input_file_path, output_directory, window_size=500, step_size=50)

### 算法实现

算法函数实现

In [12]:

def calculate_entropy(data):
    """
    计算给定数据的熵值。

    参数:
    data (pd.Series): 输入数据列。

    返回:
    float: 计算得到的熵值。
    """
    probability = data.value_counts(normalize=True)  # 计算每个值的频率
    entropy = -np.sum(probability * np.log2(probability + 1e-10))  # 加上一个小常数以避免对数为零
    return entropy

def process_directory(input_directory, label):
    """
    读取指定目录中的 CSV 文件，计算信息熵，并生成表现 DataFrame。

    参数:
    input_directory (str): 输入 CSV 文件的目录路径。
    label (str): 为每个记录指定的标签。

    返回:
    pd.DataFrame: 包含文件名、信息熵和标签的 DataFrame。
    """
    results = []  # 用于存储结果的列表

    # 遍历文件夹中的所有 CSV 文件
    for filename in os.listdir(input_directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_directory, filename)
            df = pd.read_csv(file_path)

            # 计算信息熵
            entropy_value = calculate_entropy(df['arbitration_id'])

            # 添加结果
            results.append({
                # 'filename': filename,
                'entropy': entropy_value,
                'label': label
            })

    # 创建结果 DataFrame
    result_df = pd.DataFrame(results)
    return result_df


使用上述函数，读取最新数据为df

In [22]:
# 使用示例
input_directory = r"./data523/can0_100s_train/normal/"  # 输入文件夹路径
label = 0  # 指定标签
train_ndf = process_directory(input_directory, label)
input3_directory = r"./data523/can0_100s_train/replay/"  # 输入文件夹路径replay
# input3_directory = r"./data523/can0_100s_train/ddos/"  # 输入文件夹路径ddos
label3 = 1  # 指定标签
train_ddf = process_directory(input3_directory, label3)
train_df = pd.concat([train_ndf, train_ddf], ignore_index=True)
# 输出结果 DataFrame
print(train_df)

      entropy  label
0    4.858543      0
1    4.855972      0
2    4.872005      0
3    4.890206      0
4    4.749810      0
5    4.836217      0
6    4.852956      0
7    4.862879      0
8    4.868756      0
9    4.745567      0
10   4.861931      0
11   4.867196      0
12   4.887937      0
13   4.860699      0
14   4.851972      0
15   4.868869      0
16   4.732328      0
17   4.847991      0
18   4.861049      0
19   4.875167      0
20   4.868193      0
21   4.793554      0
22   4.751671      0
23   4.832294      0
24   4.888705      0
25   4.866429      0
26   4.835205      0
27   4.873385      0
28   4.715029      0
29   4.858082      0
30   4.859348      0
31   4.896561      0
32   4.947022      0
33   4.878605      0
34   4.766487      0
35   4.835950      0
36   4.893956      0
37   4.901062      0
38   4.879152      0
39   4.878620      0
40   4.723297      0
41   4.869222      0
42   4.859286      0
43   4.911816      0
44   4.889255      0
45   4.898483      0
46   4.772340

读取test数据

In [23]:
# 使用示例
input2_directory = r"./data523/can0_500s_test/normal/"  # 输入文件夹路径
label = 0  # 指定标签
test_ndf = process_directory(input2_directory, label)
input4_directory = r"./data523/can0_500s_test/replay/"  # 输入文件夹路径replay
# input4_directory = r"./data523/can0_500s_test/ddos/"  # 输入文件夹路径ddos
label3 = 1  # 指定标签
test_ddf = process_directory(input4_directory, label3)
test_df = pd.concat([test_ndf, test_ddf], ignore_index=True)
# 输出结果 DataFrame
print(test_df)

     entropy  label
0   4.791866      0
1   4.920468      0
2   4.908160      0
3   4.902282      0
4   4.909256      0
5   4.742627      0
6   4.872054      0
7   4.903213      0
8   4.900224      0
9   4.925253      0
10  4.702794      0
11  4.872004      0
12  4.767074      0
13  4.952594      0
14  4.930730      0
15  4.939788      0
16  4.949360      0
17  4.804560      0
18  4.836821      0
19  4.923933      0
20  4.911731      0
21  4.913355      0
22  4.943949      0
23  4.938890      0
24  4.753333      0
25  4.903053      0
26  4.872767      0
27  4.856565      0
28  4.889291      0
29  4.838245      0
30  4.724923      0
31  4.898414      0
32  4.920076      0
33  4.876662      0
34  4.890779      0
35  4.908896      0
36  4.769492      0
37  4.853222      0
38  4.939140      0
39  4.907651      0
40  4.912687      0
41  4.914591      0
42  4.756411      0
43  4.887118      0
44  4.896813      0
45  4.892597      0
46  4.886866      0
47  4.899017      0
48  4.838227      0


#### 使用正态分布假设进行验证

In [30]:
# 从训练集提取特征和标签
X_train = train_df['entropy'].values  # 训练集特征数组
y_train = train_df['label'].values    # 训练集标签数组

# 从测试集提取特征和标签
X_test = test_df['entropy'].values    # 测试集特征数组
y_test = test_df['label'].values      # 测试集标签数组

# Step 1: 划分训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: 在正常样本上拟合高斯模型
normal_data = X_train[y_train == 0]
mu, sigma = np.mean(normal_data), np.std(normal_data)

# Step 3: 计算训练集的异常分数
cdf_values = norm.cdf(X_train, mu, sigma) 
tail_probs = np.where(X_train < mu, cdf_values, 1 - cdf_values)
two_tailed_p_values = 2 * tail_probs
anomaly_scores_train = 1 - two_tailed_p_values

# Step 4: 计算测试集的异常分数
cdf_values_test = norm.cdf(X_test, mu, sigma)
tail_probs_test = np.where(X_test < mu, cdf_values_test, 1 - cdf_values_test)
two_tailed_p_values_test = 2 * tail_probs_test
anomaly_scores_test = 1 - two_tailed_p_values_test

# Step 5: 计算 ROC 曲线并选择最佳阈值
fpr, tpr, thresholds = roc_curve(y_train, anomaly_scores_train)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
# optimal_threshold = 0.6

# Step 6: 在测试集上进行预测
y_pred = (anomaly_scores_test > optimal_threshold).astype(int)  # 预测标签

# 输出结果
print("测试集的预测标签:", y_pred)
print(mu)
print(sigma)
print(optimal_threshold)



# 假设 y_pred 是您的预测结果，y 是原始标签
# y_pred = [0, 1, 0, 1, ...]
# y = [0, 1, 0, 0, ...]

# 计算准确度
accuracy = accuracy_score(y_test, y_pred)
print(f"准确度: {accuracy:.2f}")

# 计算混淆矩阵
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

# 计算误检率（假阳性率）
false_positive_rate = fp / (fp + tn)
print(f"误检率: {false_positive_rate:.4f}")
recall = tp / (tp + fn)
print(f"Recall: {recall:.4f}")

测试集的预测标签: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 